# ピクトグラムをスクレイピング

- http://pictogram2.com/
- https://qiita.com/neet-AI/items/98d4194872ee4f53e3b4

In [21]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import re

In [18]:
# もとURLのスクレイピング
BASE_URL = "http://pictogram2.com/?paged={}"

results = []
for num_page in range(1, 41):
    url = BASE_URL.format(num_page)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    page_urls = [item.get("href") for item in soup.find_all("a", attrs={"href": re.compile("http://pictogram2.com/\?p=\d+")})]
    results.extend(page_urls)
    time.sleep(1)

In [23]:
# 保存
pd.DataFrame([[url] for url in results], columns=["url"]).to_csv("../data/pict_urls.csv", index=False)

In [37]:
# 画像をダウンロード

for url in results:
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    imgs = soup.find_all('img',src=re.compile('/p/p.+jpg'))
    for img in imgs:
        r = requests.get("http://pictogram2.com/"+img['src'])
        with open(str('../picture/')+img['src'].replace("/", "_"),'wb') as file:
            file.write(r.content)
        time.sleep(0.5)